In [32]:
# !pip3 install scikit-image

In [2]:
import kagglehub
path = kagglehub.dataset_download("ankit1743/skyview-an-aerial-landscape-dataset") + "/Aerial_Landscapes"
path

/Users/bytedance/Library/Python/3.9/lib/python/site-packages/urllib3/__init__.py:35: NotOpenSSLWarning: urllib3 v2 only supports OpenSSL 1.1.1+, currently the 'ssl' module is compiled with 'LibreSSL 2.8.3'. See: https://github.com/urllib3/urllib3/issues/3020
  warnings.warn(
/Users/bytedance/Library/Python/3.9/lib/python/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


'/Users/bytedance/.cache/kagglehub/datasets/ankit1743/skyview-an-aerial-landscape-dataset/versions/1/Aerial_Landscapes'

# machine learning

## color

In [10]:
import os
import cv2
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report

def extract_color_histogram(image, bins=(16, 16, 16)):
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    hist = cv2.calcHist([hsv], [0, 1, 2], None, bins,
                        [0, 180, 0, 256, 0, 256])
    cv2.normalize(hist, hist)
    return hist.flatten()

dataset_path = path  

color_features = []
labels = []

for label in os.listdir(dataset_path):
    folder = os.path.join(dataset_path, label)
    if not os.path.isdir(folder):
        continue  
    
    for img_file in os.listdir(folder):
        img_path = os.path.join(folder, img_file)
        try:
            image = cv2.imread(img_path)
            if image is None:
                continue
            image = cv2.resize(image, (100, 100))  
            hist = extract_color_histogram(image)
            color_features.append(hist)
            labels.append(label)
        except Exception as e:
            print(f"error here {img_path}: {e}")

x_train, x_test, y_train, y_test = train_test_split(
    color_features, labels, test_size=0.2, random_state=42)


knn model

In [11]:
from sklearn.neighbors import KNeighborsClassifier

knn_model = KNeighborsClassifier(n_neighbors=5) 
knn_model.fit(x_train, y_train)

print("knn accuracy:", knn_model.score(x_test, y_test))
print(classification_report(y_test, knn_model.predict(x_test)))

knn accuracy: 0.5933333333333334
              precision    recall  f1-score   support

 Agriculture       0.44      0.48      0.46       168
     Airport       0.38      0.37      0.38       157
       Beach       0.62      0.40      0.49       156
        City       0.63      0.74      0.68       170
      Desert       0.74      0.94      0.83       170
      Forest       0.64      0.91      0.75       161
   Grassland       0.64      0.85      0.73       152
     Highway       0.58      0.24      0.34       173
        Lake       0.60      0.28      0.38       165
    Mountain       0.52      0.66      0.59       155
     Parking       0.65      0.71      0.67       157
        Port       0.61      0.56      0.59       149
     Railway       0.55      0.64      0.59       163
 Residential       0.67      0.80      0.73       148
       River       0.56      0.34      0.42       156

    accuracy                           0.59      2400
   macro avg       0.59      0.60      0.57    

svm

In [12]:
from sklearn.svm import SVC

svm_model = SVC(kernel='rbf', random_state=42) 
svm_model.fit(x_train, y_train)

print("svm accuracy:", svm_model.score(x_test, y_test))
print(classification_report(y_test, svm_model.predict(x_test)))

svm accuracy: 0.685
              precision    recall  f1-score   support

 Agriculture       0.58      0.54      0.56       168
     Airport       0.47      0.54      0.50       157
       Beach       0.66      0.71      0.69       156
        City       0.80      0.75      0.78       170
      Desert       0.85      0.95      0.90       170
      Forest       0.83      0.88      0.85       161
   Grassland       0.72      0.89      0.80       152
     Highway       0.66      0.44      0.53       173
        Lake       0.66      0.48      0.56       165
    Mountain       0.67      0.62      0.64       155
     Parking       0.72      0.75      0.73       157
        Port       0.63      0.74      0.68       149
     Railway       0.62      0.72      0.67       163
 Residential       0.83      0.79      0.81       148
       River       0.57      0.50      0.53       156

    accuracy                           0.69      2400
   macro avg       0.68      0.69      0.68      2400
weight

forest

In [14]:
from sklearn.ensemble import RandomForestClassifier

forest_model = RandomForestClassifier(n_estimators=100, random_state=42)
forest_model.fit(x_train, y_train)

print("forest accuracy:", forest_model.score(x_test, y_test))
print(classification_report(y_test, forest_model.predict(x_test)))

forest accuracy: 0.7191666666666666
              precision    recall  f1-score   support

 Agriculture       0.66      0.69      0.67       168
     Airport       0.54      0.59      0.56       157
       Beach       0.72      0.80      0.76       156
        City       0.71      0.74      0.72       170
      Desert       0.92      0.96      0.94       170
      Forest       0.81      0.90      0.85       161
   Grassland       0.87      0.93      0.90       152
     Highway       0.68      0.45      0.54       173
        Lake       0.64      0.52      0.57       165
    Mountain       0.66      0.65      0.65       155
     Parking       0.83      0.80      0.82       157
        Port       0.81      0.77      0.79       149
     Railway       0.61      0.65      0.63       163
 Residential       0.74      0.86      0.80       148
       River       0.56      0.52      0.54       156

    accuracy                           0.72      2400
   macro avg       0.72      0.72      0.72 

## texture

### lbp

In [15]:
from skimage import feature

def extract_lbp_features(image, num_points=24, radius=8, eps=1e-7):
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    lbp = feature.local_binary_pattern(gray, num_points, radius, method="uniform")
    (hist, _) = np.histogram(lbp.ravel(),
                             bins=np.arange(0, num_points + 3),
                             range=(0, num_points + 2))
    hist = hist.astype("float")
    hist /= (hist.sum() + eps) 
    return hist

texture_features = []
labels = []

for label in os.listdir(dataset_path):
    folder = os.path.join(dataset_path, label)
    if not os.path.isdir(folder):
        continue  

    for img_file in os.listdir(folder):
        img_path = os.path.join(folder, img_file)
        try:
            image = cv2.imread(img_path)
            if image is None:
                continue
            image = cv2.resize(image, (100, 100))
            lbp_hist = extract_lbp_features(image)
            texture_features.append(lbp_hist)
            labels.append(label)
        except Exception as e:
            print(f"Error processing {img_path}: {e}")

x_train, x_test, y_train, y_test = train_test_split(
    texture_features, labels, test_size=0.2, random_state=42)

knn_model.fit(x_train, y_train)
print("LBP - KNN Accuracy:", knn_model.score(x_test, y_test))

svm_model.fit(x_train, y_train)
print("LBP - SVM Accuracy:", svm_model.score(x_test, y_test))

forest_model.fit(x_train, y_train)
print("LBP - RF Accuracy:", forest_model.score(x_test, y_test))


LBP - KNN Accuracy: 0.3229166666666667
LBP - SVM Accuracy: 0.20208333333333334
LBP - RF Accuracy: 0.40125
